In [2]:
import numpy as np
import xgboost as xgb
import pandas as pd
import category_encoders as ce
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.utils.multiclass import unique_labels
import csv

/Users/jarroyo/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/jarroyo/miniforge3/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
train_data = pd.read_csv('../data/LOANS_TRAIN.csv')
test_data = pd.read_csv('../data/LOANS_TEST.csv')

id_column = train_data

train_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)
test_data.drop(columns=['id','grade', 'emp_title', 'title'], axis=1, inplace=True)

#Strip percent(%) from int_rate
train_data['int_rate'] = train_data['int_rate'].str.rstrip('%').astype(float)
test_data['int_rate'] = test_data['int_rate'].str.rstrip('%').astype(float)

#Strip percent(%) from revol_util
train_data['revol_util'] = train_data['revol_util'].str.rstrip('%').astype(float)
test_data['revol_util'] = test_data['revol_util'].str.rstrip('%').astype(float)

X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]
X_test = test_data.iloc[:,:]

X_train_numeric = X_train.select_dtypes(include=np.number)
X_test_numeric = X_train.select_dtypes(include=np.number)
# y_train_numeric = y_train.select_dtypes(include=np.number)
y_train_numeric = y_train.copy(deep=False)
y_train_numeric.replace('Fully Paid', 0, inplace=True)
y_train_numeric.replace('Charged Off', 1, inplace=True)
print(X_train_numeric.shape)


(197250, 13)


In [10]:
X_balanced = []
Y_balanced = []
for i in range(X_train_numeric.shape[0]):
    curr_df = X_train_numeric.iloc[i,:]
    d = curr_df.to_dict()
    label = y_train_numeric[i]
    if label == 1:
        for _ in range(3):
            X_balanced.append(d)
            Y_balanced.append(label)
    X_balanced.append(d)
    Y_balanced.append(label)
X_balanced = pd.DataFrame(X_balanced)
Y_balanced = pd.Series(Y_balanced)

print("BEFORE")
print(X_train_numeric.shape)
print(y_train_numeric.shape)
print('AFTER')
print(X_balanced.shape)
print(Y_balanced.shape)
# print(X_balanced)
# Y_balanced = pd.Series(dtype=float)
# print(y_train_numeric.shape)
# print("BEFORE")
# print(X_train_numeric[1])
# for k, row in X_train_numeric.iterrows():
#     label = pd.Series([y_train_numeric[k]])
#     if y_train_numeric[k] == 1:
#         for _ in range(3):
#             X_balanced.append(row)
#             Y_balanced.append(label)
#     X_balanced.append(row)
#     Y_balanced.append(label)
#     if k == 10:
#         break
# for k, row in X_train_numeric.iterrows():
#     label = pd.Series([y_train_numeric[k]])
#     print(type(row))
#     if y_train_numeric[k] == 1:
#         for _ in range(3):
#             X_balanced = pd.concat([X_balanced, row.to_frame()])
#             Y_balanced = pd.concat([Y_balanced, label])
#     X_balanced = pd.concat([X_balanced, row.to_frame()])
#     Y_balanced = pd.concat([Y_balanced, label])
#     if k == 10:
#         break
# print("AFTER")
# print(X_balanced.shape)
# print(Y_balanced.shape)

BEFORE
(197250, 13)
(197250,)
AFTER
(287922, 13)
(287922,)


In [11]:
print(X_balanced)
print(Y_balanced)
X_tr, X_val, Y_tr, Y_val = sklearn.model_selection.train_test_split(
    X_train_numeric, y_train_numeric, test_size=0.2, random_state=0)

        loan_amnt  term_(months)  int_rate  installment  annual_inc    dti  \
0          5000.0           36.0     10.65       162.87     24000.0  27.65   
1          2500.0           60.0     15.27        59.83     30000.0   1.00   
2          2500.0           60.0     15.27        59.83     30000.0   1.00   
3          2500.0           60.0     15.27        59.83     30000.0   1.00   
4          2500.0           60.0     15.27        59.83     30000.0   1.00   
...           ...            ...       ...          ...         ...    ...   
287917    31400.0           60.0     22.95       884.28     85000.0  24.76   
287918     5500.0           36.0     14.09       188.22     20000.0  29.28   
287919     5500.0           36.0     14.09       188.22     20000.0  29.28   
287920     5500.0           36.0     14.09       188.22     20000.0  29.28   
287921     5500.0           36.0     14.09       188.22     20000.0  29.28   

        open_acc  pub_rec  revol_bal  revol_util  total_acc  mo

In [45]:
def auc(m, xtrain, xtest): 
    return (metrics.roc_auc_score(Y_tr,m.predict_proba(xtrain)[:,1]),
                            metrics.roc_auc_score(Y_val, m.predict_proba(xtest)[:,1]))

# Parameter Tuning
# model = xgb.XGBClassifier()
# param_dist = {"max_depth": [10,30,50],
#               "min_child_weight" : [1,3,6],
#               "n_estimators": [5],
#               "learning_rate": [0.05, 0.1,0.16],}
# grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
#                                    verbose=10, n_jobs=-1)
# grid_search.fit(X_tr, Y_tr)

# grid_search.best_estimator_

model = xgb.XGBClassifier(max_depth=12, min_child_weight=1,  n_estimators=15,\
                          n_jobs=-1,learning_rate=0.01, gamma=5)
model.fit(X_tr,Y_tr)
predictions = model.predict_proba(X_val)
print(type(predictions))
print(sklearn.metrics.roc_auc_score(Y_val, predictions[:,1]))
print('HER')
auc(model, X_tr, X_val)


/Users/jarroyo/miniforge3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/jarroyo/miniforge3/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[15:59:43] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1643226991592/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
<class 'numpy.ndarray'>
0.673049507425129
HER


/Users/jarroyo/miniforge3/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/Users/jarroyo/miniforge3/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


(0.7197867384892856, 0.673049507425129)

In [ ]:
predictions = {"id": [], "loan_status": []}
for index, value in X_test.items():
    credit_score = row["grade"]
    idx = grade_2_index[credit_score]
    feature = [0 for _ in range(7)]
    feature[idx] = 1
    prediction = model.predict_proba([feature])[0][1]
    predictions["id"].append(row['id'])
    predictions["loan_status"].append(prediction)